![](./img/chinahadoop.png)
# 风控实战项目 -- 电信行业风控建模
**[小象学院](http://www.chinahadoop.cn/course/landpage/15)《机器学习集训营》实战项目案例 by [@寒小阳](http://www.chinahadoop.cn/user/49339/about)**

## 说明
本notebook为特征工程部分，给出了基本的思路和框架，请大家填充完相应环节的步骤，同时也鼓励大家尝试更多更好的数据处理与特征工程和建模方式。

### 引入工具库

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import xgboost as xgb

咱们从单表和多表交叉，分别抽取单维度特征和交叉维度特征

### wa features
1. 访问网站的中的个数， wa_all_cnt
2. 不同网站的次数， wa_all_unique_cnt
3. visit_cnt的，sum, mean, std, max, min, skew
4. visit_dura
5. up_flow
6. down_flow
7. down_flow - up_flow = down_up_flow_diff
8. wa_type, 分布统计
9. date的分布个数，以及对应的不同次数，up_flow, down_flow, visit_dura, visit_cnt的统计值

In [3]:
df_train_wa = pd.read_csv('../data/train/wa_train.txt', names = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', \
                                   'up_flow', 'down_flow', 'wa_type', 'date'], sep='\t', low_memory=False)

df_testA_wa = pd.read_csv('../data/testA/wa_test_a.txt', names = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', \
                                   'up_flow', 'down_flow', 'wa_type', 'date'], sep='\t',low_memory=False)

df_testB_wa = pd.read_csv('../data/testB/wa_test_b.txt', names = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', \
                                   'up_flow', 'down_flow', 'wa_type', 'date'], sep='\t',low_memory=False)

df_train_label = pd.read_csv('../data/train/uid_train.txt', names = ['uid', 'label'], sep='\t', low_memory=False)

In [ ]:
wa_name_risk = list(set(df_train_wa[df_train_wa['uid'].isin(df_train_label[df_train_label['label']==1]['uid'])]['wa_name']))

In [ ]:
def get_wa_feature_plus(df_train_wa, target='train', Type=None):
    if target == 'train':
        # 复制lable的数据，作为所有的特征的标示
        df_train = df_train_label.copy()
    else:
        if Type == 'A':
            df_train = pd.DataFrame(data={'uid':['u'+str(id) for id in range(5000, 7000)]})
        else:
            df_train = pd.DataFrame(data={'uid':['u'+str(id) for id in range(7000, 10000)]})
    
    # wa次数特征
    # your code here
    
    
    
    # top高频的wa_name特征
    # your code here
    
    
    
    # top高频的wa_name分布特征
    # your code here
    
    
    
    # 平均wa_name的个数， wa_all_per_opp_rate
    # your code here
    
            
        
    # wa_type 分布
    # your code here
    
    
    
    # in_out 的比例
    # your code here
    
    
    
    # visit_cnt, visit_dura, up_flow, down_flow, down_up_flow_diff, stats_values
    # 统计值(sum mean max min std skew)
    for feat in ['visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'down_up_flow_diff','visit_dura_div_visit_cnt','up_flow_div_visit_cnt','down_flow_div_visit_cnt']:
        # your code here

    
    # 上述得到的交叉特征
    for feat in ['sum','avg','max','min','std','skew']:
        # your code here
    
    
    # 全局top的wa_name在当前样本的统计值
    # your code here
    
    
    
    # 根据date进行分类的统计值，以及比例
    # your code here
    
    
    
    # 其他补充特征
    # your code here

    
    return df_train

In [ ]:
# 功能函数：时间差，计算两个时间点秒数的差别
def diff_time(a,b):
    a_day, a_hour, a_minute, a_second = (a / 1000000, a / 10000 % 100, a / 100 % 100, a % 100)
    b_day, b_hour, b_minute, b_second = (b / 1000000, b / 10000 % 100, b / 100 % 100, b % 100)
    
    d_day = b_day - a_day
    d_hour = b_hour - a_hour
    d_minute = b_minute - a_minute
    d_second = b_second - a_second
    
    diff = d_day * 24 * 60 * 60 + d_hour * 60 * 60 + d_minute * 60 + d_second
    return diff

def get_diff_time(x):
    diff_t = []
    for d in x:
        diff_t.append(diff_time(d[0],d[1]))
    return diff_t

### 调用特征工程处理

In [ ]:
df_test = get_wa_feature_plus(df_testB_wa, mode='test', Type='B')
df_testA = get_wa_feature_plus(df_testA_wa, mode='test', Type='A')
df_train = get_wa_feature_plus(df_train_wa)

df_train['wa_all_wa_name_little_wite_risk'] = df_train['wa_all_wa_name_little_wite'].isin(wa_name_risk)
df_train['wa_all_wa_name_many_wite_risk'] = df_train['wa_all_wa_name_many_wite'].isin(wa_name_risk)
df_test['wa_all_wa_name_little_wite_risk'] = df_test['wa_all_wa_name_little_wite'].isin(wa_name_risk)
df_test['wa_all_wa_name_many_wite_risk'] = df_test['wa_all_wa_name_many_wite'].isin(wa_name_risk)

df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)


df_train.to_csv('./features/df_train_wa_feat.csv',index=False)
df_test.to_csv('./features/df_testB_wa_feat.csv',index=False)
df_testA.to_csv('./features/df_testA_wa_feat.csv',index=False)

### 查看数据

In [ ]:
df_train.info()
df_test.info()
df_test.head()